In [1]:
using DataFrames, Arrow, UMAP, StatsBase, MLJ

In [2]:
df = DataFrame(Arrow.Table("L:/laupodteam/AIOS/Bram/data/CellDyn/celldyn_cleaned_transformed_imputed_MCAR_with_ratios.feather");)

,level_0,index,studyid_alle_celldyn,bestanden
,Int64?,Int64?,Float64?,Array…?
1,0,55192,2.75292e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
2,1,55194,3.34401e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
3,2,55195,5.43972e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
4,3,55197,2.72134e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
5,4,55198,2.04489e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
6,5,55199,2.00961e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
7,6,55200,5.72492e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
8,7,55201,2.04489e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"
9,8,55202,3.95235e9,"[0x31, 0x5f, 0x30, 0x39, 0x30, 0x35, 0x45, 0x2e, 0x63, 0x73, 0x76]"


┌ Warning: automatically converting Arrow.Timestamp with precision = NANOSECOND to `Dates.DateTime` which only supports millisecond precision; conversion may be lossy; to avoid converting, pass `Arrow.Table(source; convert=false)
└ @ Arrow C:\Users\hjoosse3\.julia\packages\Arrow\ZlMFU\src\eltypes.jl:261


In [3]:
meas_cols = names(df)[contains.(names(df),"c_b_")];
suspect_cols = names(df)[contains.(names(df),r"^(c_)?s_")];
mode_cols = names(df)[contains.(names(df),"mode")];
alrt_cols = names(df)[contains.(names(df),"Alrt")];
other_c_cols = names(df)[contains.(names(df),r"^c_")];
usual_cols = unique(vcat(meas_cols,suspect_cols,mode_cols,alrt_cols,other_c_cols));

In [4]:
import Distances

struct PoinCarre <: Distances.SemiMetric end

poincarre(x, y) = PoinCarre()(x, y)

function (dist::PoinCarre)(x,y)
    return acosh(1 + 2*(norm(x-y,2)^2/((1-norm(x,2)^2)*(1-norm(yu,2)^2))
    ))
end

@inline (dist::PoinCarre)(a::AbstractArray, b::AbstractArray) = Distances._evaluate(dist, a, b)

In [5]:
X = df[:,meas_cols]
X = dropmissing(X)
X_coerced = coerce(X, autotype(X))
dt = StatsBase.fit(ZScoreTransform, Matrix(X_coerced); dims=nothing, center=true, scale=true)
X_transformed = StatsBase.transform(dt,Matrix(X_coerced));

In [6]:
x_embedded = umap(transpose(X_transformed),6;n_neighbors=50,min_dist = 0.1)

┌ Info: SuiteSparse.CHOLMOD.CHOLMODException("out of memory")
│ Error encountered in spectral_layout; defaulting to random layout
└ @ UMAP C:\Users\hjoosse3\.julia\packages\UMAP\owv5b\src\embeddings.jl:12


6×2241484 Matrix{Float64}:
 -3.07185     -2.37643   -1.92611  …   1.09321   -3.56134    -2.34237
 -1.6387      -2.09341    2.1139      -3.23372   -0.277035   -2.1296
 -2.36664     -3.13598    3.21748     -4.04549   -0.325142   -2.68138
  0.823456     1.11138   -1.12779      0.610087   0.852549    0.60331
  0.00167751  -0.185471   1.44027     -0.536295   0.399219   -0.657713
  1.69146      2.31715   -2.54449  …   3.73973   -0.0862846   2.04178